In [2]:
import json
import string

import wikipedia
import wptools
from SPARQLWrapper import SPARQLWrapper, JSON
from nltk import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize

In [13]:
catType = {"Airports": "?a a dbo:Infrastructure .", "Comics_characters": "?a a dbo:Agent .",
           "Artist": "?a a dbo:Animal .",
           "Astronauts": '?a a dbo:Animal .', "Building": "?a a dbo:Building .", "Astronomical_objects": "",
           "City": "?a a dbo:Place .", "Companies": "?a a dbo:Company .", "Foods": "?a a dbo:Food .",
           "Transport": "?a a dbo:MeanOfTransportation .", "Monuments_and_memorials": "?a a dbo:Place .",
           "Politicians": "?a a dbo:Animal .", "Sports_teams": "?a a dbo:Organisation .",
           "Sportspeople": "?a a dbo:Animal .", "Universities_and_colleges": "?a a dbo:Organisation .",
           "Written_communication": ""}
stop_words = stopwords.words('english')
porter = PorterStemmer()
sparql_dbpedia = SPARQLWrapper("http://dbpedia.org/sparql")
sparql_dbpedia.setReturnFormat(JSON)
sparql_wd = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql_wd.setReturnFormat(JSON)

In [4]:
k = 10
n = 2

## Exercise 1 :

In [ ]:
data = []
for cat in catType:
    SPARQL_GET_LISTS = f'''
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX dbp: <http://dbpedia.org/property/>
    PREFIX dbc: <http://dbpedia.org/resource/Category:>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX dct: <http://purl.org/dc/terms/>

    SELECT DISTINCT ?page ?p ?code WHERE {{
    ?a dct:subject/skos:broader dbc:{cat} .
    {catType[cat]}
    ?a rdfs:label ?label .
    ?a foaf:name ?page .
    ?p foaf:primaryTopic ?a .
    ?a dbo:wikiPageID ?code .
    FILTER (lang(?label) = 'en')
    FILTER(!STRSTARTS(?label, "List of"))
    }}limit {k}
    '''
    sparql_dbpedia.setQuery(SPARQL_GET_LISTS)
    try:
        ret = sparql_dbpedia.queryAndConvert()
        for r in ret["results"]["bindings"]:
            url_name = r["code"]['value']  #re.findall("([^/]+$)", url)[0]
            page = wikipedia.page(pageid=url_name)
            cont = page.content
            number_of_sentences = len(sent_tokenize(cont))
            if number_of_sentences >= n:
                p = wptools.page(pageid=url_name, silent=True)
                p.get_parse()
                p.get_wikidata()
                data.append(
                    {"name": r["page"]['value'], "url_name": url_name, "txt": page.content,
                     "infobox": p.data["infobox"],
                     "wikidata": p.data["wikidata"],
                     "cat": cat, "url": r["p"]['value'], "wikibase": p.data["wikibase"]})
    except Exception as e:
        print()

with open("data.json", "w") as f:
    json.dump(data, f)


## Exercise 2 :


In [3]:
file = open("data.json", "r")
data_to_process = json.load(file)

In [16]:
def preprocess(txt):
    text = txt.lower()
    text_p = "".join([char for char in text if char not in string.punctuation])
    words = word_tokenize(text_p)
    filtered_words = [word for word in words if word not in stop_words]
    stemmed = [porter.stem(word) for word in filtered_words]
    pos = pos_tag(stemmed)
    return pos, stemmed


def getDescription(item):
    SPARQL_DESCR = f'''
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX schema: <http://schema.org/>
    SELECT ?o WHERE {{
    wd:{item} schema:description ?o .
    FILTER ( lang(?o) = "en" )
    }}
    '''
    sparql_wd.setQuery(SPARQL_DESCR)
    try:
        res = sparql_wd.queryAndConvert()
        print(res)
        return res["results"]["bindings"][0]["o"]["value"]
    except Exception as e:
        print(e)


for item in data_to_process:
    txt = preprocess(item["txt"])
    desc = getDescription("Q47472325")
    wikidata = preprocess(desc)

    break

{'head': {'vars': ['o']}, 'results': {'bindings': [{'o': {'xml:lang': 'en', 'type': 'literal', 'value': 'request for comments publication'}}]}}
